In [18]:
import json
import os
import pandas as pd
import boto3
from sqlalchemy import create_engine, text
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
import plotly.express as px


In [19]:
df_hotels = pd.read_json(r'/Users/ymilo/Desktop/Jedha/home/Scrappy_prpject/kayak_spiders/Hotels_dataset.json')

In [25]:
#df_hotels = pd.read_csv('/Users/ymilo/Desktop/Jedha/home/Scrappy_prpject/df_hostels.csv')
df_cities = pd.read_csv('/Users/ymilo/Desktop/Jedha/home/Scrappy_prpject/df_cities.csv')

In [26]:
del df_cities['Unnamed: 0']

In [27]:
df_cities = df_cities.rename(columns={'lat' : 'lat_location', 'lon' : 'lon_location'})


In [22]:
del df_hotels['Unnamed: 0']

KeyError: 'Unnamed: 0'

# Select warmest cities in cities dataframe

In [28]:
df_cities = df_cities.sort_values('Forecast', ascending=False).head()

# Rank Hotels in location

In [29]:
def rank(x):
    lst_rank = []
    x = x.sort_values("score",ascending=False)
    for i in range(len(x)):
        lst_rank.append(i+1)
    x["rank"] = lst_rank
    return x

In [30]:
loc_names = df_hotels.location.unique()
data_list = []
full_data_list = []
for i in loc_names:
    data_list.append(df_hotels[df_hotels['location'] == i])
for i in range(len(data_list)):
    full_data_list.append(rank(data_list[i]))
df_hotels = pd.concat(full_data_list)

In [31]:
df_hotels = df_hotels[df_hotels["rank"] <= 20]

# Merge and upload datasets

In [32]:
df_full = pd.merge(df_hotels, df_cities, on='location' )

In [33]:
df_full.to_csv('/Users/ymilo/Desktop/Jedha/home/Scrappy_prpject/df_full.csv')

In [35]:
ACCESS_KEY_ID = "REDACTED" # cle du compte student
SECRET_ACCESS_KEY = "REDACTED" # secret key du compte student
session = boto3.Session(aws_access_key_id=ACCESS_KEY_ID, 
                        aws_secret_access_key=SECRET_ACCESS_KEY)

In [36]:
s3 = session.resource("s3")

In [37]:
bucket = s3.Bucket("kayak-003")

In [38]:
bucket.upload_file('/Users/ymilo/Desktop/Jedha/home/Scrappy_prpject/df_full.csv', 'df_full.csv')

# Downaload file from S3 to confirm proper uplaod and run SQL database

In [39]:
bucket.download_file('df_full.csv', 'df_extract.csv')

In [40]:
df_extracted = pd.read_csv('/Users/ymilo/Desktop/Jedha/home/Scrappy_prpject/df_extract.csv')

In [41]:
engine = create_engine("sqlite:///:memory:", echo=True)

In [42]:
df_full.to_sql(
    "df_sql_full",
    engine
)

2023-02-15 15:40:58,734 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("df_sql_full")
2023-02-15 15:40:58,735 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-15 15:40:58,737 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("df_sql_full")
2023-02-15 15:40:58,738 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-15 15:40:58,740 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-15 15:40:58,744 INFO sqlalchemy.engine.Engine 
CREATE TABLE df_sql_full (
	"index" BIGINT, 
	name TEXT, 
	location TEXT, 
	description TEXT, 
	score FLOAT, 
	url TEXT, 
	lat FLOAT, 
	lon FLOAT, 
	rank BIGINT, 
	lat_location FLOAT, 
	lon_location FLOAT, 
	"Forecast" FLOAT
)


2023-02-15 15:40:58,747 INFO sqlalchemy.engine.Engine [no key 0.00366s] ()
2023-02-15 15:40:58,749 INFO sqlalchemy.engine.Engine CREATE INDEX ix_df_sql_full_index ON df_sql_full ("index")
2023-02-15 15:40:58,750 INFO sqlalchemy.engine.Engine [no key 0.00080s] ()
2023-02-15 15:40:58,751 INFO sqlalchemy.engine.Engine COMMIT
202

100

In [43]:
df_full.columns

Index(['name', 'location', 'description', 'score', 'url', 'lat', 'lon', 'rank',
       'lat_location', 'lon_location', 'Forecast'],
      dtype='object')

In [44]:
stmt = text("SELECT DISTINCT location FROM df_sql_full")

In [45]:
df_from_sql = pd.read_sql(stmt, engine)

2023-02-15 15:41:00,541 INFO sqlalchemy.engine.Engine SELECT DISTINCT location FROM df_sql_full
2023-02-15 15:41:00,557 INFO sqlalchemy.engine.Engine [generated in 0.01615s] ()


In [46]:
df_from_sql

,location
0,Aigues Mortes
1,Saintes Maries de la mer
2,Marseille
3,Collioure
4,Cassis


# Make maps

In [47]:
fig = px.scatter_mapbox(df_full, lat=df_full["lat_location"].unique(), lon=df_full["lon_location"].unique(), color=df_full["Forecast"].unique(), size=df_full["Forecast"].unique())

fig.update_layout(mapbox_style="open-street-map")

In [48]:
fig_2 = px.scatter_mapbox(df_full, lat="lat", lon="lon", color="score")
fig_2.update_layout(mapbox_style="open-street-map")